<a href="https://colab.research.google.com/github/marongjun/risk_evaluation/blob/master/fintech_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

df_tag = pd.read_csv('./train_tag.csv')

In [0]:
# store column id as training labels
train_labels = df_tag['flag']
train_labels.to_csv('labels.csv')

In [0]:
#clean data: fill null value and '\N' with flag value -999
df_tag = df_tag.replace('\\N', -999)
df_tag = df_tag.fillna(-999)
df_tag.drop(['id','flag'],axis=1, inplace=True)

# One hot encoding categorical variables
df_tag = pd.concat([df_tag, pd.get_dummies(data=df_tag,
                                           columns=['gdr_cd','mrg_situ_cd','edu_deg_cd','acdm_deg_cd','deg_cd'],
                                           prefix=['gender','marrige', 'education','academic_degree','degree']
                                           )],axis=1)
df_tag.drop(['gdr_cd','mrg_situ_cd','edu_deg_cd','acdm_deg_cd','deg_cd'],axis=1, inplace=True)

#check if still any null value exists, convert all datatypes to numeric, then check datatypes
df_tag.isnull().sum
train_data = df_tag.apply(pd.to_numeric)
train_data.dtypes

#save training matrix
train_data.to_csv('train_data.csv')

In [0]:
import tensorflow as tf
from tensorflow import keras

layers = tf.keras.layers

model_mlp = tf.keras.models.Sequential()
model_mlp.add(layers.Dense(256, activation=tf.nn.relu, input_shape=(train_data.shape[1],)))
model_mlp.add(layers.Dropout(0.2))
model_mlp.add(layers.Dense(128, activation=tf.nn.relu))
model_mlp.add(layers.Dropout(0.2))
model_mlp.add(layers.Dense(128, activation=tf.nn.relu))
model_mlp.add(layers.Dropout(0.2))
model_mlp.add(layers.Dense(2, activation=tf.nn.softmax))
model_mlp.compile(optimizer = 'adam',
           loss = 'sparse_categorical_crossentropy',
           metrics=['accuracy'])

In [141]:
model_mlp.fit(train_data, train_labels, batch_size=64, epochs=10, shuffle=True)

Epoch 1/10
624/624 [==============================] - 2s 3ms/step - loss: 4.6560 - accuracy: 0.6955
Epoch 2/10
624/624 [==============================] - 2s 3ms/step - loss: 0.6161 - accuracy: 0.7664
Epoch 3/10
624/624 [==============================] - 2s 3ms/step - loss: 0.5533 - accuracy: 0.7735
Epoch 4/10
624/624 [==============================] - 2s 3ms/step - loss: 0.5411 - accuracy: 0.7745
Epoch 5/10
624/624 [==============================] - 2s 3ms/step - loss: 0.5396 - accuracy: 0.7752
Epoch 6/10
624/624 [==============================] - 2s 3ms/step - loss: 0.5337 - accuracy: 0.7754
Epoch 7/10
624/624 [==============================] - 2s 3ms/step - loss: 0.5339 - accuracy: 0.7754
Epoch 8/10
624/624 [==============================] - 2s 3ms/step - loss: 0.5470 - accuracy: 0.7753
Epoch 9/10
624/624 [==============================] - 2s 3ms/step - loss: 0.5359 - accuracy: 0.7755
Epoch 10/10
624/624 [==============================] - 2s 3ms/step - loss: 0.5328 - accuracy: 0.7756

In [148]:
import pandas as pd

df_tag_test = pd.read_csv('./evaluate_tag.csv')
test_id = df_tag_test['id']

#clean data: fill null value and '\N' with flag value -999
df_tag_test  = df_tag_test .replace('\\N', -999)
df_tag_test  = df_tag_test .fillna(-999)
df_tag_test .drop(['id'],axis=1, inplace=True)

# One hot encoding categorical variables
df_tag_test  = pd.concat([df_tag_test , pd.get_dummies(data = df_tag_test ,
                                           columns=['gdr_cd','mrg_situ_cd','edu_deg_cd','acdm_deg_cd','deg_cd'],
                                           prefix=['gender','marrige', 'education','academic_degree','degree']
                                           )],axis=1)
df_tag_test .drop(['gdr_cd','mrg_situ_cd','edu_deg_cd','acdm_deg_cd','deg_cd'],axis=1, inplace=True)

#check if still any null value exists, convert all datatypes to numeric, then check datatypes
df_tag_test .isnull().sum
test_data = df_tag_test .apply(pd.to_numeric)
test_data.dtypes
print(test_data.shape)

(6000, 108)


In [149]:
test_data.insert(88, "education_J", 0) 
train_column = list(train_data.columns)
test_column = list(test_data.columns)
print(train_data.columns.get_loc("education_J"))
print(train_column)
print(test_column)
print(test_data.shape)

88
['age', 'job_year', 'ic_ind', 'fr_or_sh_ind', 'dnl_mbl_bnk_ind', 'dnl_bind_cmb_lif_ind', 'hav_car_grp_ind', 'hav_hou_grp_ind', 'l6mon_agn_ind', 'frs_agn_dt_cnt', 'vld_rsk_ases_ind', 'fin_rsk_ases_grd_cd', 'confirm_rsk_ases_lvl_typ_cd', 'cust_inv_rsk_endu_lvl_cd', 'l6mon_daim_aum_cd', 'tot_ast_lvl_cd', 'pot_ast_lvl_cd', 'bk1_cur_year_mon_avg_agn_amt_cd', 'l12mon_buy_fin_mng_whl_tms', 'l12_mon_fnd_buy_whl_tms', 'l12_mon_insu_buy_whl_tms', 'l12_mon_gld_buy_whl_tms', 'loan_act_ind', 'pl_crd_lmt_cd', 'ovd_30d_loan_tot_cnt', 'his_lng_ovd_day', 'hld_crd_card_grd_cd', 'crd_card_act_ind', 'l1y_crd_card_csm_amt_dlm_cd', 'atdd_type', 'perm_crd_lmt_cd', 'cur_debit_cnt', 'cur_credit_cnt', 'cur_debit_min_opn_dt_cnt', 'cur_credit_min_opn_dt_cnt', 'cur_debit_crd_lvl', 'age', 'job_year', 'ic_ind', 'fr_or_sh_ind', 'dnl_mbl_bnk_ind', 'dnl_bind_cmb_lif_ind', 'hav_car_grp_ind', 'hav_hou_grp_ind', 'l6mon_agn_ind', 'frs_agn_dt_cnt', 'vld_rsk_ases_ind', 'fin_rsk_ases_grd_cd', 'confirm_rsk_ases_lvl_typ_cd',

In [150]:
predictions = model_mlp.predict(test_data)
pred = pd.DataFrame(predictions)
print(pred)

            0        1
0     0.77134  0.22866
1     0.77134  0.22866
2     0.77134  0.22866
3     0.77134  0.22866
4     0.77134  0.22866
...       ...      ...
5995  0.77134  0.22866
5996  0.77134  0.22866
5997  0.77173  0.22827
5998  0.77134  0.22866
5999  0.77134  0.22866

[6000 rows x 2 columns]


In [152]:
result = pd.concat([test_id,pred[0]],axis=1)
print(result)
result.to_csv('predictions.csv')

           id        0
0     UC37930  0.77134
1     U5BE130  0.77134
2     UD025AE  0.77134
3     UC2D00D  0.77134
4     UAF705D  0.77134
...       ...      ...
5995  U71611F  0.77134
5996  U52CFE4  0.77134
5997  U6A5425  0.77173
5998  UB8871F  0.77134
5999  U5573FB  0.77134

[6000 rows x 2 columns]


In [0]:
import numpy as np
np.savetxt(r'result.txt', result.values, fmt='%s')